# Подготовка данных для CAD

## Парсинг файла

In [1]:
import os
import time
import pandas as pd

from tqdm import tqdm

from pathlib import Path
from typing import List, Dict, Any, Optional, Union

In [2]:
import httpx
import json
import asyncio
import codecs

In [3]:
EMBEDDING_MODEL_NAME = "D:\\models\\e5-lagre"

### Локальное подключение модели Qwen3 30B A3B через LM Studio

In [4]:
from openai import OpenAI

MODEL_CONFIG = {
    "base_url": "http://172.21.16.126:1234/v1",
    "api_key": "not-needed",
    "model_name": "yandexgpt-5-lite-8b-instruct", #"qwen/qwen3-30b-a3b",
    "temperature": 0.1,
    "top_p": 0.9
}

In [5]:
llm = OpenAI(base_url=MODEL_CONFIG["base_url"], api_key=MODEL_CONFIG["api_key"])

In [6]:
prompt = "Напиши последовательно числа от 1 до 10"
no_think = "/no_think\n"

messages = []
messages.append({"role": "system", "content": "Ты знаток арифметики"})
messages.append({"role": "user", "content": no_think+prompt})

reply = llm.chat.completions.create(
    model=MODEL_CONFIG["model_name"],
    messages=messages,
    temperature=MODEL_CONFIG["temperature"],
    top_p=MODEL_CONFIG["top_p"],
    stream=False
)

response = reply.choices[0].message.content
print("Response:",response)

Response:  1, 2, 3, 4, 5, 6, 7, 8, 9, 10.


In [7]:
class LLM:
    def __init__(self):
        self.llm = OpenAI(base_url=MODEL_CONFIG["base_url"], api_key=MODEL_CONFIG["api_key"])
        self.system_prompt = "Ты выполняеешь задание пользователя. Не добавляй лишних слов, выводи только результат"

    def set_system_prompt(self, system_prompt: str):
        """Установка системного промпта"""
        self.system_prompt = system_prompt

    def generate_answer(self, question: str) -> str:
        """Генерация ответа модели"""

        # no_think = "/no_think\n"
        messages = []
        messages.append({"role": "system", "content": self.system_prompt})
        messages.append({"role": "user", "content": no_think + question})
        
        reply = self.llm.chat.completions.create(
            model=MODEL_CONFIG["model_name"],
            messages=messages,
            temperature=MODEL_CONFIG["temperature"],
            top_p=MODEL_CONFIG["top_p"],
            stream=False
        )
        
        return reply.choices[0].message.content #[19:]

In [8]:
llm = LLM()

In [9]:
llm.generate_answer("Напиши последовательно числа от 1 до 10")

' 1, 2, 3, 4, 5, 6, 7, 8, 9, 10.'

In [10]:
llm.generate_answer("Ответь на Главный вопрос Жизни, Вселенной и всего такого")

' 42.'

## Парсинг и подготовка

In [11]:
from docx import Document
from docx.text.paragraph import Paragraph

BORDER = 7400
MAX_TOKENS = 8192
TEMPERATURE = 0.2
TOP_P = 0.9

In [12]:
import nest_asyncio
nest_asyncio.apply()

In [13]:
class DOCXParser:
    """
    Класс для парсинга DOCX файлов
    """

    def __init__(self, server_url):
        self.document = pd.DataFrame(columns=['Header','Text', 'CleanText', 'HTML'])
        self.llm = LLM()
        self.server_url = server_url

    def parse_docx(self, file_path: str) -> pd.DataFrame:
        """
        Парсит DOCX файл и возвращает DataFrame с содержимым файла
        """

        print(f"Парсинг документа: {file_path}")

        doc = Document(file_path)
        tab_cnt = 0
        curr_par_len = 0
        chapters = []

        lvl = 0
        number_arr = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

        for el in doc.element.body:
            if el.tag.endswith('p'):

                paragraph = Paragraph(el, doc)
                style = paragraph.style.name

                if "Heading" in style:

                    hd_parts = style.split()
                    curr_lvl = int(hd_parts[-1]) - 1
                    number_arr[curr_lvl] += 1

                    # Проверим на заполнение номеров предыдущих уровней
                    for level in range(curr_lvl):
                        if not number_arr[level]:
                            number_arr[level] = 1
                    
                    number = ""
                    for i in range(curr_lvl+1):
                        if number:
                            number += "."
                        number += str(number_arr[i])

                    while curr_lvl < lvl:
                        number_arr[lvl] = 0
                        lvl -= 1

                    lvl = curr_lvl
                    
                    print(f"Добавлена глава: {number} {el.text.strip()}")
                    # print("  "*curr_lvl + number + " " + el.text.strip())

                    curr_par_len = 0

                    header = el.text.strip()
                    if type(header) == type(0.0):
                        header = ""
                    chapters.append([number + " " + header, ""])

                elif paragraph.text:
                    curr_par_len += len(paragraph.text)
                    if len(chapters):
                        chapters[-1][-1] += paragraph.text
                    else:
                        chapters.append([paragraph.text[:20], paragraph.text]) # текст до первого заголовка

            elif el.tag.endswith('tbl'):
                tab_cnt += 1

                table_data = []

                if not len(chapters):
                    # Вариант 1
                    for row in doc.tables[tab_cnt-1].rows:
                        row_data = [cell.text.strip() for cell in row.cells]
                        table_data.append(row_data)

                else:
                    # Вариант 2
                    chapters[-1][-1] += "\n-----\n"
                    for row in doc.tables[tab_cnt-1].rows:
                        chapters[-1][-1] += "|"
                        for cell in row.cells:
                            chapters[-1][-1] += " " + cell.text.strip() + " |"
                        chapters[-1][-1] += "\n"
                    chapters[-1][-1] += "-----\n"                    
                
                if len(chapters):
                    chapters[-1][-1] += str(table_data)
                else:
                    chapters.append([str(table_data)[:20], str(table_data)])    # таблица до первого заголовка

        for chapter in chapters:
            if len(chapter[1]) > 1:
                new_row = {'Header': chapter[0], 'Text': chapter[1], 'CleanText': '', 'HTML': ''}
                self.document = pd.concat([self.document,
                                           pd.DataFrame([new_row],
                                           columns=self.document.columns)],
                                           ignore_index = True)

        return self.document

    def clean_text(self) -> pd.DataFrame:
        """
        Очищает текст с помощью БЯМ
        """

        data_size = self.document.shape[0]

        self.llm.set_system_prompt("Ты профессионально работаешь с текстом и выполняешь роль корректора. " \
                                   "Исправляй опечатки, восстанавливай пропущенные пробелы и пунктуацию. " \
                                   "Не добавляй лишних слов, выводи только результат")

        for i in range(data_size):
            if self.document.iloc[i]["Text"]:

                prompt = f"Исправь опечатки в документе. Вставть пробелы где необходимо. " \
                        f"Удали служебные символы, например: '\xa0'. Выведи только исправленный текст документа. " \
                        f"Документ:\n\n{self.document.iloc[i]['Text'][:BORDER]}"

                self.document.loc[i, 'CleanText'] = self.llm.generate_answer(prompt)

                print(f"Отредактирован раздел {i+1}: {self.document.loc[i, 'CleanText'][:50]}")
                    
        return self.document

    def make_html(self) -> pd.DataFrame:
        """
        Создаёт HTML разметку глав документа
        """

        data_size = self.document.shape[0]

        html_sample = '<!DOCTYPE html>\n<html lang="ru">\n<head>\n    <meta charset="UTF-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <title>Путеводитель по Китаю</title>\n    <style>\n        body {\n            font-family: Arial, sans-serif;\n            line-height: 1.6;\n            margin: 20px;\n            color: #333;\n            font-size: 18px;\n        }\n        h1 {\n            color: #2c3e50;\n            font-size: 32px;\n        }\n        h2 {\n            color: #2980b9;\n            font-size: 28px;\n        }\n        p {\n            margin-bottom: 15px;\n        }\n        .highlight {\n            background-color: #f9e79f;\n            font-weight: bold;\n        }\n    </style>\n</head>\n<body>\n    <header>\n        <h1>Путеводитель по Китаю</h1>\n    </header>\n\n    <main>\n        <section>\n            <h2>Авиаперевозки</h2>\n            <p>В Китае насчитывается 980 линий внутренних воздушных перевозок, 130 международных и 24 линии региональных. Узловым пунктом в сети международных авиалиний является Пекин.</p>\n        </section>\n\n        <section>\n            <h2>Железнодорожное сообщение</h2>\n            <p>В стране развито железнодорожное сообщение. Пассажиров обслуживают как поезда внутренней службы, так и поезда международных перевозок. Внутренние поезда бывают скоростные, скорые, туристические, экспрессы и скорые поезда прямого сообщения.</p>\n        </section>\n\n</body>\n</html>'

        self.llm.set_system_prompt("Ты профессионально создаёшь страницы в формате HTML")
        
        for i in range(data_size):
            if self.document.iloc[i]["CleanText"]:

                header = self.document.iloc[i]["Header"]
                text = self.document.iloc[i]["CleanText"]

                prompt = f"Сгенерируй HTML разметку для отображения страницы. Важно: Полностью используй ТЕКСТ ГЛАВЫ, не сокращай! " \
                         f"Используй ПРИМЕР РАЗМЕТКИ. Не выводи ничего кроме разметки. " \
                         f"Заголовок страницы: '{header}'. ПРИМЕР РАЗМЕТКИ: {html_sample} ТЕКСТ ГЛАВЫ: {text}"

                self.document.loc[i, 'HTML'] = self.llm.generate_answer(prompt)

                print(f"{i+1} Сгенерирована HTML разметка: {self.document.loc[i, 'HTML'][:15]}")

        return self.document

    def get_document(self) -> pd.DataFrame:
        """
        Геттер - возвращает DataFrame с содержимым документа
        """
        return self.document

    def load(self, file_name: str) -> None:
        """
        Загружаем документ (Pandas DataFrame) из CSV файла
        """
        self.document = pd.read_csv(file_name, sep=";")

    def save(self, file_name: str) -> None:
        """
        Сохраняем документ (Pandas DataFrame) в CSV файл
        """
        self.document.to_csv(file_name, sep=";", index=False)

    def _generate_ids_sync(self, count: int) -> List[str]:
        """
        Генерируем уникальные Id на сервере для имён файлов с HTML разметкой
        """
        try:
            response = httpx.post(
                f"{self.server_url}/generate_ids",
                json={"count": count},
                timeout=30.0
            )
            res = response.json()
            return res.get("ids", [])
        except Exception as e:
            print(f"Ошибка генерации ID: {e}")
            return [f"id-{i}" for i in range(count)]

    def create_html_files(self, path_html_pages: str) -> None:
        """
        Сохраняем сгенерированную HTML разметку в файлы с уникальными именами
        """

        html_id_list = self._generate_ids_sync(len(self.document))
        print(f"Сгенерировано {len(html_id_list)} уникальных Id")

        if not os.path.exists(path_html_pages):
            os.makedirs(path_html_pages)
        
        data["HTML_name"] = ""
        i = 0
        for j in range(data.shape[0]):
            if data["HTML"][j] != "-" and len(data["HTML"][j]):
                file_path = os.path.join(path_html_pages, f"{html_id_list[i]}.html")
                self.document.loc[j, 'HTML_name'] = f"{html_id_list[i]}.html"
                with codecs.open(file_path, "w", "utf-8-sig") as file:
                    if data["HTML"][j][:3] == "```":
                        file.write(data["HTML"][j][7:-3])
                    else:
                        file.write(data["HTML"][j])
                    data.iloc[j, -1] = f"{html_id_list[i]}.html"
                    print(f"{i+1} Сохранён файл: {data.iloc[j, -1]}")
                    i += 1

    def create_txt_files(self, path_txt_pages: str) -> None:
        """
        Сохраняем сохраняем извлечённый текст в файлы с именами - заголовками
        """

        if not os.path.exists(path_txt_pages):
            os.makedirs(path_txt_pages)

        for i in range(data.shape[0]):
            if data["Text"][i] != "-" and len(data["Text"][i]):
                file_name = data['Header'][i].replace(',', '').replace('/', '').replace('\\', '')
                file_path = os.path.join(path_txt_pages, f"{file_name}.txt")

                with codecs.open(file_path, "w", "utf-8-sig") as file:
                    file.write(data["Text"][i])
                    print(f"{i+1} Сохранён файл: {file_name}.txt")
                    i += 1

In [152]:
parser = DOCXParser("http://172.21.16.126:9090")

In [81]:
start = time.time()

data = parser.parse_docx("./merged_cr.docx")

end = time.time()
print(f"Время исполнения: {(end - start):.3f} с.")

Парсинг документа: ./merged_cr.docx
Добавлена глава: 1 Назначение программного модуля CAD
Добавлена глава: 2 Условия выполнения программного модуля CAD
Добавлена глава: 3 Выполнение программного модуля CAD
Добавлена глава: 3.1 Запуск программного модуля CAD
Добавлена глава: 3.2 Управление приложениями
Добавлена глава: 3.3 Работа с документами
Добавлена глава: 3.3.1 Создание нового документа
Добавлена глава: 3.3.2 Открытие документов
Добавлена глава: 3.3.3 Сохранение документов
Добавлена глава: 3.3.4 Закрытие документа
Добавлена глава: 3.3.5 Настройка документов
Добавлена глава: 3.3.5.1 Свойства документа
Добавлена глава: 3.3.5.2 Атрибуты документа
Добавлена глава: 3.3.5.3 История документа
Добавлена глава: 3.3.5.4 Стили объектов
Добавлена глава: 3.4 Справочная система
Добавлена глава: 4 Интерфейс
Добавлена глава: 4.1 Окна программного модуля CAD
Добавлена глава: 4.1.1 Рабочие окна
Добавлена глава: 4.1.2 Вспомогательные окна
Добавлена глава: 4.2 Инструментальные панели
Добавлена глава: 

In [83]:
data.shape

(208, 4)

In [85]:
data.head()

,Header,Text,CleanText,HTML
0,1 Назначение программного модуля CAD,Программный модуль CAD разработан на базе инте...,,
1,2 Условия выполнения программного модуля CAD,Достаточными условиями выполнения программного...,,
2,3.1 Запуск программного модуля CAD,Загрузка и запуск программного модуля CAD осущ...,,
3,3.2 Управление приложениями,Данное окно предназначено для осуществления уп...,,
4,3.3 Работа с документами,Перечень инструментов для работы с документами...,,


In [87]:
data.tail()

,Header,Text,CleanText,HTML
203,12.9 Проверка модели,Команда «Проверка модели» предназначена для то...,,
204,13.1 Экспорт и импорт 3D-геометрии,Команды «Экспорт» и «Импорт» предназначены д...,,
205,13.2 Печать,Команда «Печать» вызывает стандартный для опе...,,
206,13.3 Экспорт PDF,Команда «Сохранить PDF» в меню «Файл» предназ...,,
207,13.4 Экспорт изображения,Команда «Экспорт изображения» в меню «Файл» п...,,


In [89]:
start = time.time()

parser.clean_text()

end = time.time()
print(f"Время исполнения: {(end - start):.3f} с.")

Отредактирован раздел 1: Программный модуль CAD разработан на базе интегрир
Отредактирован раздел 2: Достаточными условиями выполнения программного мод
Отредактирован раздел 3: Загрузка и запуск программного модуля CAD осуществ
Отредактирован раздел 4: Данное окно предназначено для осуществления управл
Отредактирован раздел 5: Перечень инструментов для работы с документами пре
Отредактирован раздел 6: Команда «Новый документ» предназначена для создани
Отредактирован раздел 7: Команда «Открыть» вызывает стандартный диалог выбо
Отредактирован раздел 8: Команда «Сохранить» выполняет сохранение текущего 
Отредактирован раздел 9: Для закрытия документа нажмите Х на вкладке рядом 
Отредактирован раздел 10: Свойства активного документа отображаются во вспом
Отредактирован раздел 11: Атрибутом документа называется переменная, помечен
Отредактирован раздел 12: Система регистрирует все действия по изменению сос
Отредактирован раздел 13: Под стилем следует понимать предустановленный набо
Отредакт

In [36]:
parser.load("2025_07_24 CAD prep.csv")

In [40]:
parser.get_document().head()

,Header,Text,CleanText,HTML,HTML_name
0,1 Назначение программного модуля CAD,Программный модуль CAD разработан на базе инте...,Программный модуль CAD разработан на базе инте...,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",f488d75c-0d69-4d67-a0be-578d5137c768.html
1,2 Условия выполнения программного модуля CAD,Достаточными условиями выполнения программного...,Достаточными условиями выполнения программного...,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",759d06d8-3cc8-4814-9094-6bbe8381095d.html
2,3.1 Запуск программного модуля CAD,Загрузка и запуск программного модуля CAD осущ...,Загрузка и запуск программного модуля CAD осущ...,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",6f300c6a-3d34-46b1-b123-123c21aeb761.html
3,3.2 Управление приложениями,Данное окно предназначено для осуществления уп...,Данное окно предназначено для осуществления уп...,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",725124d4-cfb3-4dfa-b176-cd85db1c0898.html
4,3.3 Работа с документами,Перечень инструментов для работы с документами...,Перечень инструментов для работы с документами...,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",ffae8eee-7309-426a-9ccd-3a4c2b74b1e9.html


In [155]:
start = time.time()

parser.make_html()

end = time.time()
print(f"Время исполнения: {(end - start):.3f} с.")

1 Сгенерирована HTML разметка: <!DOCTYPE html>
2 Сгенерирована HTML разметка: <!DOCTYPE html>
3 Сгенерирована HTML разметка: <!DOCTYPE html>
4 Сгенерирована HTML разметка: <!DOCTYPE html>
5 Сгенерирована HTML разметка: <!DOCTYPE html>
6 Сгенерирована HTML разметка: <!DOCTYPE html>
7 Сгенерирована HTML разметка: <!DOCTYPE html>
8 Сгенерирована HTML разметка: <!DOCTYPE html>
9 Сгенерирована HTML разметка: <!DOCTYPE html>
10 Сгенерирована HTML разметка: <!DOCTYPE html>
11 Сгенерирована HTML разметка: <!DOCTYPE html>
12 Сгенерирована HTML разметка: <!DOCTYPE html>
13 Сгенерирована HTML разметка: <!DOCTYPE html>
14 Сгенерирована HTML разметка: <!DOCTYPE html>
15 Сгенерирована HTML разметка: <!DOCTYPE html>
16 Сгенерирована HTML разметка: <!DOCTYPE html>
17 Сгенерирована HTML разметка: <!DOCTYPE html>
18 Сгенерирована HTML разметка: <!DOCTYPE html>
19 Сгенерирована HTML разметка: <!DOCTYPE html>
20 Сгенерирована HTML разметка: <!DOCTYPE html>
21 Сгенерирована HTML разметка: <!DOCTYPE html>
2

In [164]:
data = parser.get_document()
data.head()

,Header,Text,CleanText,HTML
0,1 Назначение программного модуля CAD,Программный модуль CAD разработан на базе инте...,Программный модуль CAD разработан на базе инте...,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ..."
1,2 Условия выполнения программного модуля CAD,Достаточными условиями выполнения программного...,Достаточными условиями выполнения программного...,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ..."
2,3.1 Запуск программного модуля CAD,Загрузка и запуск программного модуля CAD осущ...,Загрузка и запуск программного модуля CAD осущ...,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ..."
3,3.2 Управление приложениями,Данное окно предназначено для осуществления уп...,Данное окно предназначено для осуществления уп...,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ..."
4,3.3 Работа с документами,Перечень инструментов для работы с документами...,Перечень инструментов для работы с документами...,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ..."


In [166]:
data.tail()

,Header,Text,CleanText,HTML
203,12.9 Проверка модели,Команда «Проверка модели» предназначена для то...,Команда «Проверка модели» предназначена для то...,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ..."
204,13.1 Экспорт и импорт 3D-геометрии,Команды «Экспорт» и «Импорт» предназначены д...,Команды «Экспорт» и «Импорт» предназначены для...,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ..."
205,13.2 Печать,Команда «Печать» вызывает стандартный для опе...,Команда «Печать» вызывает стандартный для опер...,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ..."
206,13.3 Экспорт PDF,Команда «Сохранить PDF» в меню «Файл» предназ...,Команда «Сохранить PDF» в меню «Файл» предназн...,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ..."
207,13.4 Экспорт изображения,Команда «Экспорт изображения» в меню «Файл» п...,Команда «Экспорт изображения» в меню «Файл» пр...,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ..."


In [158]:
parser.save("2025_07_24 CAD prep.csv")

## Генерируем уникальные Id и сохраняем разметку в файлы

In [154]:
parser.load("2025_07_24 CAD prep.csv")

In [156]:
data = parser.get_document()
data.head()

,Header,Text,CleanText,HTML,HTML_name,Question
0,1 Назначение программного модуля CAD,Программный модуль CAD разработан на базе инте...,Программный модуль CAD разработан на базе инте...,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",f488d75c-0d69-4d67-a0be-578d5137c768.html,Какие функции выполняет программный модуль CAD...
1,2 Условия выполнения программного модуля CAD,Достаточными условиями выполнения программного...,Достаточными условиями выполнения программного...,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",759d06d8-3cc8-4814-9094-6bbe8381095d.html,Какие системные требования необходимо выполнит...
2,3.1 Запуск программного модуля CAD,Загрузка и запуск программного модуля CAD осущ...,Загрузка и запуск программного модуля CAD осущ...,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",6f300c6a-3d34-46b1-b123-123c21aeb761.html,Какие элементы содержатся на стартовой страниц...
3,3.2 Управление приложениями,Данное окно предназначено для осуществления уп...,Данное окно предназначено для осуществления уп...,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",725124d4-cfb3-4dfa-b176-cd85db1c0898.html,Какие действия можно выполнить с программными ...
4,3.3 Работа с документами,Перечень инструментов для работы с документами...,Перечень инструментов для работы с документами...,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",ffae8eee-7309-426a-9ccd-3a4c2b74b1e9.html,Какой раздел меню содержит перечень инструмент...


In [225]:
path_html_pages = os.path.join(os.getcwd(), "html_pages")
path_html_pages

'C:\\Users\\glvv2\\hr_assistant\\html_pages'

In [201]:
parser._generate_ids_sync(2)

['09f2880d-2918-4a4e-be00-bf090fd7c5e6',
 'f29e60f0-1544-40ac-b8e3-18247d51b0c3']

In [227]:
parser.create_html_files("html_pages")

Сгенерировано 208 уникальных Id
1 Сохранён файл: f488d75c-0d69-4d67-a0be-578d5137c768.html
2 Сохранён файл: 759d06d8-3cc8-4814-9094-6bbe8381095d.html
3 Сохранён файл: 6f300c6a-3d34-46b1-b123-123c21aeb761.html
4 Сохранён файл: 725124d4-cfb3-4dfa-b176-cd85db1c0898.html
5 Сохранён файл: ffae8eee-7309-426a-9ccd-3a4c2b74b1e9.html
6 Сохранён файл: 165dfda8-be0b-4a10-a3c0-ae40d86e654e.html
7 Сохранён файл: e01bda90-ab2b-4d7b-805f-f17101e368fe.html
8 Сохранён файл: 8e52cb7f-9f96-4d82-8ce1-c8b4fb5f9932.html
9 Сохранён файл: 564977cd-a507-4ab7-a902-6ba5bed05373.html
10 Сохранён файл: 2206ec5d-9a54-40fd-8b06-e4d72e7b5b0e.html
11 Сохранён файл: 0ce13713-da46-4a52-b109-2a570d7393c9.html
12 Сохранён файл: 9db792c2-60ce-4d27-a130-a58a4a9dbd12.html
13 Сохранён файл: bba6b7dc-a7d0-4839-b731-8cd4e8dadb2d.html
14 Сохранён файл: 1738c668-9383-4bfe-a4ed-cce85d668bd9.html
15 Сохранён файл: ce2a7f9a-019b-41c2-a855-5be7dcbe4f33.html
16 Сохранён файл: a399cf21-025b-4b4c-b500-162c16e04ade.html
17 Сохранён файл:

In [229]:
parser.save("2025_07_24 CAD prep.csv")

In [231]:
parser.get_document().tail()

,Header,Text,CleanText,HTML,HTML_name
203,12.9 Проверка модели,Команда «Проверка модели» предназначена для то...,Команда «Проверка модели» предназначена для то...,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",bf9d61da-e442-41ac-85c6-faf6f61b9984.html
204,13.1 Экспорт и импорт 3D-геометрии,Команды «Экспорт» и «Импорт» предназначены д...,Команды «Экспорт» и «Импорт» предназначены для...,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",bf8faae8-2751-4c09-b93c-581b4e110891.html
205,13.2 Печать,Команда «Печать» вызывает стандартный для опе...,Команда «Печать» вызывает стандартный для опер...,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",2444d67f-8555-4559-8f31-c1e280c5ee2a.html
206,13.3 Экспорт PDF,Команда «Сохранить PDF» в меню «Файл» предназ...,Команда «Сохранить PDF» в меню «Файл» предназн...,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",81659370-1281-4700-b465-22954a834d08.html
207,13.4 Экспорт изображения,Команда «Экспорт изображения» в меню «Файл» п...,Команда «Экспорт изображения» в меню «Файл» пр...,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",139c834b-f38f-401a-8dce-3cfa2c7f7a57.html


### Сохраняем текст в файлы для GraphRAG

In [158]:
parser.create_txt_files("txt_cad_files")

1 Сохранён файл: 1 Назначение программного модуля CAD.txt
2 Сохранён файл: 2 Условия выполнения программного модуля CAD.txt
3 Сохранён файл: 3.1 Запуск программного модуля CAD.txt
4 Сохранён файл: 3.2 Управление приложениями.txt
5 Сохранён файл: 3.3 Работа с документами.txt
6 Сохранён файл: 3.3.1 Создание нового документа.txt
7 Сохранён файл: 3.3.2 Открытие документов.txt
8 Сохранён файл: 3.3.3 Сохранение документов.txt
9 Сохранён файл: 3.3.4 Закрытие документа.txt
10 Сохранён файл: 3.3.5.1 Свойства документа.txt
11 Сохранён файл: 3.3.5.2 Атрибуты документа.txt
12 Сохранён файл: 3.3.5.3 История документа.txt
13 Сохранён файл: 3.3.5.4 Стили объектов.txt
14 Сохранён файл: 3.4 Справочная система.txt
15 Сохранён файл: 4 Интерфейс.txt
16 Сохранён файл: 4.1 Окна программного модуля CAD.txt
17 Сохранён файл: 4.1.1 Рабочие окна.txt
18 Сохранён файл: 4.1.2 Вспомогательные окна.txt
19 Сохранён файл: 4.2 Инструментальные панели.txt
20 Сохранён файл: 4.2.1 Панель «Лента».txt
21 Сохранён файл: 4.2.

## Формирование векторной базы

In [244]:
parser.load("2025_07_24 CAD prep.csv")
data = parser.get_document()
data.head()

,Header,Text,CleanText,HTML,HTML_name
0,1 Назначение программного модуля CAD,Программный модуль CAD разработан на базе инте...,Программный модуль CAD разработан на базе инте...,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",f488d75c-0d69-4d67-a0be-578d5137c768.html
1,2 Условия выполнения программного модуля CAD,Достаточными условиями выполнения программного...,Достаточными условиями выполнения программного...,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",759d06d8-3cc8-4814-9094-6bbe8381095d.html
2,3.1 Запуск программного модуля CAD,Загрузка и запуск программного модуля CAD осущ...,Загрузка и запуск программного модуля CAD осущ...,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",6f300c6a-3d34-46b1-b123-123c21aeb761.html
3,3.2 Управление приложениями,Данное окно предназначено для осуществления уп...,Данное окно предназначено для осуществления уп...,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",725124d4-cfb3-4dfa-b176-cd85db1c0898.html
4,3.3 Работа с документами,Перечень инструментов для работы с документами...,Перечень инструментов для работы с документами...,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",ffae8eee-7309-426a-9ccd-3a4c2b74b1e9.html


In [13]:
data = pd.read_csv("merged_cr3.csv", sep=";")
data.head()

,Header,Text,CleanText,HTML,Tables,Images,HTML_name
0,1 Назначение программного модуля CAD,Программный модуль CAD разработан на базе инте...,NaN,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",[],[],cf2981e0-0d00-48d2-9206-750c69b11912.html
1,2 Условия выполнения программного модуля CAD,Достаточными условиями выполнения программного...,NaN,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",[],[],fb0a33c0-9972-4761-856c-394425f09c0a.html
2,3.1 Запуск программного модуля CAD,Загрузка и запуск программного модуля CAD осущ...,NaN,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",[],"[{'id': 1, 'caption': '', 'path': WindowsPath(...",ae4eecfc-de30-4bba-879a-260ab2349ed7.html
3,3.2 Управление приложениями,\n[Изображение 2: ]\nДанное окно предназначено...,NaN,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",[],"[{'id': 2, 'caption': '', 'path': WindowsPath(...",3aef9b38-9e25-446a-ae24-3b41d655cda6.html
4,3.3 Работа с документами,Перечень инструментов для работы с документами...,NaN,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",[],[],c419d806-1a68-4943-8b85-dd0ac27a0361.html


In [14]:
import torch
from langchain_core.documents import Document
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from chromadb import PersistentClient

from langchain.text_splitter import RecursiveCharacterTextSplitter

BATCH_SIZE = 1024    # Для GPU

In [15]:
SIZE = 275

In [16]:
documents = []
metadatas = []

part_cnt = 0

for i in range(data.shape[0]):
    
    text_len = len(data.iloc[i]["Text"])
    
    if data.iloc[i]["Text"] == "-" or data.iloc[i]["Text"] != data.iloc[i]["Text"]:
        continue

    elif text_len <= SIZE:
        print(f"{i} длина: {len(data.iloc[i]['Text'])}")
        documents.append(f"{data.iloc[i]['Header'].strip()}\n{data.iloc[i]['Text'].strip()}")
        metadatas.append({"Header": data['Header'][i], 
                          "Index": str(i), 
                          "Chunk":  "",
                          "Content": data.iloc[i]["Text"].strip(),
                          # "Chapter": "",
                          "HTML_path": data.iloc[i]["HTML_name"]
                         })
        part_cnt += 1

    else:
        print(f"{i}")
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=SIZE,
            chunk_overlap=0,
            separators=["\n", " ", ""]  # "\n\n", Порядок приоритета разделителей
        )

        chunks = text_splitter.split_text(data.iloc[i]["Text"].strip())
        len_ch = len(chunks)
        step = 13
        if len_ch > step:
            for j in range(step, len_ch, step):
                part = " ".join(chunks[j-(step):j])
                for k in range(j - step, j):
                    documents.append(f"{data['Header'][i].strip()}\n{chunks[k].strip()}")
                    metadatas.append({"Header": data['Header'][i], 
                                      "Index": str(i), 
                                      "Chunk":  chunks[k],
                                      "Content": part.strip(),
                                      # "Chapter": "" # data.iloc[i]["Clean_Text"].strip(),
                                      "HTML_path": data.iloc[i]["HTML_name"]
                                     })
                    print(f"part len: {len(part)} chunk len: {len(chunks[k])}") # {chunks[k]:50}")
                part_cnt += 1
        else:
            j = 0
        
        if j < len_ch or len_ch < step:
            part = " ".join(chunks[j:])
            for k in range(j, len_ch):
                documents.append(f"{data['Header'][i].strip()}\n{chunks[k].strip()}")
                metadatas.append({"Header": data['Header'][i], 
                                  "Index": str(i), 
                                  "Chunk":  chunks[k],
                                  "Content": part.strip(),
                                  # "Chapter": "" # data.iloc[i]["Clean_Text"].strip(),
                                  "HTML_path": data.iloc[i]["HTML_name"]
                                 })
                print(f"part len: {len(part)} chunk len: {len(chunks[k])}") # {chunks[k]:50}")
            part_cnt += 1

0
part len: 620 chunk len: 209
part len: 620 chunk len: 242
part len: 620 chunk len: 167
1
part len: 374 chunk len: 257
part len: 374 chunk len: 116
2
part len: 857 chunk len: 173
part len: 857 chunk len: 243
part len: 857 chunk len: 226
part len: 857 chunk len: 212
3
part len: 1366 chunk len: 17
part len: 1366 chunk len: 271
part len: 1366 chunk len: 85
part len: 1366 chunk len: 215
part len: 1366 chunk len: 175
part len: 1366 chunk len: 168
part len: 1366 chunk len: 148
part len: 1366 chunk len: 270
part len: 1366 chunk len: 9
4 длина: 75
5
part len: 1135 chunk len: 274
part len: 1135 chunk len: 238
part len: 1135 chunk len: 271
part len: 1135 chunk len: 212
part len: 1135 chunk len: 136
6
part len: 2121 chunk len: 172
part len: 2121 chunk len: 240
part len: 2121 chunk len: 127
part len: 2121 chunk len: 273
part len: 2121 chunk len: 60
part len: 2121 chunk len: 255
part len: 2121 chunk len: 20
part len: 2121 chunk len: 48
part len: 2121 chunk len: 274
part len: 2121 chunk len: 22
par

In [17]:
documents[:5]

['1 Назначение программного модуля CAD\nПрограммный модуль CAD разработан на базе интегрированной инженерной программной платформы и программно-математического ядра трехмерного моделирования «RGK».\n\nФункциональное назначение программного модуля CAD:',
 '1 Назначение программного модуля CAD\n1. создание и редактирование геометрической модели объектов проектирования, включая 3D- и 2D-представления;\n\n\n2. задание атрибутивной информации, связанной с точностными, размерными, технологическими параметрами объектов проектирования (PMI);',
 '1 Назначение программного модуля CAD\n3. оформление графической документации в соответствии с требованиями стандартов (ЕСКД);\n\n\n4. визуализация объектов проектирования с различными параметрами отображения.',
 '2 Условия выполнения программного модуля CAD\nДостаточными условиями выполнения программного модуля CAD являются системные требования:\n\n1. ОС – Microsoft Windows версии не ниже 10, Astra Linux версии 1.7 и выше;\n\n\n2. процессор Intel Core 

In [18]:
metadatas[:3]

[{'Header': '1 Назначение программного модуля CAD',
  'Index': '0',
  'Chunk': 'Программный модуль CAD разработан на базе интегрированной инженерной программной платформы и программно-математического ядра трехмерного моделирования «RGK».\n\nФункциональное назначение программного модуля CAD:',
  'Content': 'Программный модуль CAD разработан на базе интегрированной инженерной программной платформы и программно-математического ядра трехмерного моделирования «RGK».\n\nФункциональное назначение программного модуля CAD: 1. создание и редактирование геометрической модели объектов проектирования, включая 3D- и 2D-представления;\n\n\n2. задание атрибутивной информации, связанной с точностными, размерными, технологическими параметрами объектов проектирования (PMI); 3. оформление графической документации в соответствии с требованиями стандартов (ЕСКД);\n\n\n4. визуализация объектов проектирования с различными параметрами отображения.',
  'HTML_path': 'cf2981e0-0d00-48d2-9206-750c69b11912.html'},


In [19]:
ids_list = []
try:
    response = httpx.post(
        f"http://172.21.16.126:9090/generate_ids",
        json={"count": len(documents)},
        timeout=30.0
    )
    res = response.json()
    ids_list = res.get("ids", [])
except Exception as e:
    print(f"Ошибка генерации ID: {e}")

In [20]:
ids_list[:10]

['05ecb12f-d811-49c3-b49d-39b4005d149d',
 'cb449932-772d-4544-953b-ced57f627e26',
 'dfb1a03c-89f5-424e-b1f9-6d8b6121992c',
 '95ecc4c3-5182-4b48-9945-0e5798243799',
 'cb38082f-f27e-41de-a087-b5a23d09ebfe',
 '193428ad-3ce7-4f9d-bc9c-ccb7eef0377d',
 '942a65b1-1fc6-44ad-b417-fc1e5edcc2c9',
 'd48a7e85-4b66-4d73-8ab9-0e3a87d6f12c',
 '54ac6a2f-7bf1-4088-bb54-f002e02af4cb',
 'bc5f6348-b5bc-4738-859a-655ee620aa28']

In [21]:
from dataclasses import dataclass
from enum import Enum

from pydantic import BaseModel


@dataclass
class SearchResult:
    chunk_text: str
    source: str
    page: int
    index: str
    header: str
    chunk: str
    content: str
    html_path: str
    similarity_score: float

In [22]:
torch.cuda.is_available()

True

In [23]:
class ChromaStorage:
    """
    Класс для работы с векторным хранилищем ChromaDB.
    Обеспечивает добавление, удаление и поиск векторов с использованием GPU-ускорения.
    """

    def __init__(self, db_path: str, collection_name: str = "vectors"):
        """
        Инициализирует хранилище ChromaDB.

        Args:
            db_path (str): Путь к векторной базе
        
            collection_name (str): Название коллекции для работы (по умолчанию "vectors")
        """
        self.collection_name = collection_name
        # self.config = Config()
        
        # Настройка путей
        parent_folder = Path(os.getcwd()) # Path(__file__).parent.parent
        self.model_path = "D:\models\e5-lagre" # parent_folder / "model" / "e5-lagre"
        self.db_path = db_path
        
        # Проверка доступности GPU
        self.device = self._get_best_device()
        
        # Инициализация единственной embedding функции
        self._init_embedding_function()
        
        # Инициализация клиента и векторного хранилища
        self._init_chroma_store()

        # self.config.logger.info(
        #     f"Инициализация успешна", 
        #     extra={"service": self.config.service}
        # )
        print(f"Инициализация успешна")

    def _get_best_device(self) -> str:
        """Определяет оптимальное устройство для вычислений (GPU/CPU)."""
        if torch.cuda.is_available():
            # self.config.logger.info(
            #     f"GPU доступен: {torch.cuda.get_device_name(0)}", 
            #     extra={"service": self.config.service}
            # )
            print(f"GPU доступен: {torch.cuda.get_device_name(0)}")
            return "cuda"
        else:
            # self.config.logger.warning(
            #     "GPU недоступен, используется CPU", 
            #     extra={"service": self.config.service}
            # )
            print("GPU недоступен, используется CPU")
            return "cpu"

    def _init_embedding_function(self):
        """Инициализирует единственную функцию эмбеддингов."""
        try:
            # Создаём только одну функцию эмбеддингов
            self.embedding_function = HuggingFaceEmbeddings(
                model_name=str(self.model_path),
                model_kwargs={'device': self.device},
                encode_kwargs={
                    'normalize_embeddings': True,
                    'convert_to_tensor': True,
                    'batch_size': BATCH_SIZE 
                },
                show_progress=False
            )
            
            # self.config.logger.info(
            #     f"Embedding функция инициализирована на {self.device}", 
            #     extra={"service": self.config.service}
            # )
            print(f"Embedding функция инициализирована на {self.device}")
            
        except Exception as e:
            # self.config.logger.error(
            #     f"Ошибка инициализации embedding функции: {e}", 
            #     extra={"service": self.config.service}
            # )
            print(f"Ошибка инициализации embedding функции: {e}")
            raise

    def _init_chroma_store(self):
        """Инициализирует персистентного клиента ChromaDB и векторное хранилище."""
        try:
            # Создаём клиента
            self.client = PersistentClient(path=str(self.db_path))
            
            # Инициализация векторного хранилища с единственной embedding функцией
            self.vector_store = Chroma(
                client=self.client,
                collection_name=self.collection_name,
                embedding_function=self.embedding_function,
                collection_metadata={"hnsw:space": "cosine"}
            )
            
            # Получаем коллекцию для прямых операций
            self.collection = self.vector_store._collection
            
            # self.config.logger.info(
            #     f"Chroma хранилище инициализировано для коллекции '{self.collection_name}'", 
            #     extra={"service": self.config.service}
            # )
            print(f"Chroma хранилище инициализировано для коллекции '{self.collection_name}'")
            
        except Exception as e:
            # self.config.logger.error(
            #     f"Ошибка инициализации Chroma хранилища: {e}", 
            #     extra={"service": self.config.service}
            # )
            print(f"Ошибка инициализации Chroma хранилища: {e}")
            raise

    def add_vectors(
            self,
            ids: List[str],
            texts: List[str],
            metadata: Optional[List[Dict[str, Any]]] = None
    ) -> None:
        """
        Добавляет тексты и их векторные представления в хранилище.

        Args:
            ids (List[str]): Уникальные идентификаторы записей
            texts (List[str]): Тексты для векторизации и хранения
            metadata (Optional[List[Dict]]): Метаданные для каждой записи

        Raises:
            ValueError: При некорректных входных данных
            RuntimeError: При ошибках добавления
        """
        # self.config.logger.info(
        #     "Начато добавление векторов в хранилище", 
        #     extra={"service": self.config.service}
        # )
        print("Начато добавление векторов в хранилище")
        
        # Проверка входных данных
        if not ids or not texts:
            error_msg = "IDs и тексты не могут быть пустыми"
            # self.config.logger.error(error_msg, extra={"service": self.config.service})
            print(error_msg)
            raise ValueError(error_msg)

        if len(ids) != len(texts):
            error_msg = "Длина списков ids и texts должна совпадать"
            # self.config.logger.error(error_msg, extra={"service": self.config.service})
            print(error_msg)
            raise ValueError(error_msg)
            
        if metadata is not None and len(metadata) != len(ids):
            error_msg = "Длина metadata должна совпадать с длиной ids"
            # self.config.logger.error(error_msg, extra={"service": self.config.service})
            print(error_msg)
            raise ValueError(error_msg)

        # Подготовка метаданных
        processed_metadata = []
        for i, text in enumerate(texts):
            base_meta = {
                            "source": "", 
                            "page": -1,
                            "text": text,
                            "index": metadata[i].get("Index", -1),
                            "header": metadata[i].get("Header", -1),
                            "chunk": metadata[i].get("Chunk", -1),
                            "content": metadata[i].get("Content", -1),
                            "html_path": metadata[i].get("HTML_path", -1)
                        }
            if metadata and i < len(metadata):
                base_meta.update(metadata[i])
            processed_metadata.append(base_meta)

        try:
            # Создаём документы для LangChain
            documents = [
                Document(page_content=text, metadata=meta)
                for text, meta in zip(texts, processed_metadata)
            ]
            
            # Пакетная обработка для больших объемов
            for i in range(0, len(documents), BATCH_SIZE):
                batch_docs = documents[i:i+BATCH_SIZE]
                batch_ids = ids[i:i+BATCH_SIZE]
                
                # Используем векторное хранилище для добавления
                self.vector_store.add_documents(
                    documents=batch_docs,
                    ids=batch_ids
                )
            
            # self.config.logger.info(
            #     f"Успешно добавлено {len(ids)} векторов в хранилище", 
            #     extra={"service": self.config.service}
            # )
            print(f"Успешно добавлено {len(ids)} векторов в хранилище")
            
        except Exception as e:
            error_msg = f"Ошибка при добавлении векторов в хранилище: {str(e)}"
            # self.config.logger.error(error_msg, extra={"service": self.config.service})
            print(error_msg)
            raise RuntimeError(error_msg)

    def delete_vectors(self, ids: List[str]) -> None:
        """
        Удаляет векторы по их идентификаторам.

        Args:
            ids (List[str]): Список ID для удаления

        Raises:
            ValueError: При пустом списке ID
            RuntimeError: При ошибках удаления
        """
        if not ids:
            error_msg = "Список IDs не может быть пустым"
            # self.config.logger.error(error_msg, extra={"service": self.config.service})
            print(error_msg)
            raise ValueError(error_msg)
            
        try:
            # self.config.logger.info(
            #     f"Начато удаление {len(ids)} векторов из хранилища", 
            #     extra={"service": self.config.service}
            # )
            print(f"Начато удаление {len(ids)} векторов из хранилища")
            
            # Используем метод векторного хранилища
            self.vector_store.delete(ids=ids)
            
            # self.config.logger.info(
            #     f"Векторы {ids} успешно удалены из хранилища", 
            #     extra={"service": self.config.service}
            # )
            print(f"Векторы {ids} успешно удалены из хранилища")
            
        except Exception as e:
            error_msg = f"Ошибка при удалении векторов из хранилища: {str(e)}"
            # self.config.logger.error(error_msg, extra={"service": self.config.service})
            print(error_msg)
            raise RuntimeError(error_msg)
        
    def get_embeddings(self, texts: List[str]) -> List[List[float]]:
        """
        Генерация эмбеддингов для списка текстов.

        Args:
            texts (List[str]): Список текстов для преобразования в эмбеддинги

        Returns:
            List[List[float]]: Список векторов эмбеддингов

        Raises:
            ValueError: При пустом списке текстов или пустых строках
            RuntimeError: При ошибках генерации эмбеддингов
        """
        if not texts:
            return []
        
        try:
            # self.config.logger.info(
            #     f"Начато создание эмбеддингов для {len(texts)} текстов", 
            #     extra={"service": self.config.service}
            # )
            print(f"Начато создание эмбеддингов для {len(texts)} текстов")
            
            # Используем единственную embedding функцию
            embeddings = self.embedding_function.embed_documents(texts)
            
            # self.config.logger.info(
            #     f"Успешно созданы эмбеддинги для {len(texts)} текстов. "
            #     f"Размерность: {len(embeddings[0]) if embeddings else 'N/A'}", 
            #     extra={"service": self.config.service}
            # )
            print(f"Успешно созданы эмбеддинги для {len(texts)} текстов. ")
            
            return embeddings
            
        except Exception as e:
            error_msg = f"Ошибка при создании эмбеддингов: {str(e)}"
            # self.config.logger.error(error_msg, extra={"service": self.config.service})
            print(error_msg)
            raise RuntimeError(error_msg)

    def search(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[SearchResult]:
        """
        Выполняет семантический поиск по запросу.

        Args:
            query (str): Поисковый запрос
            top_k (int): Количество возвращаемых результатов (по умолчанию 5)
            score_threshold (float): Порог схожести (0.0-1.0)

        Returns:
            List[SearchResult]: Результаты поиска

        Raises:
            ValueError: При пустом запросе
            RuntimeError: При ошибках поиска
        """
        if not query.strip():
            error_msg = "Запрос не может быть пустым"
            # self.config.logger.error(error_msg, extra={"service": self.config.service})
            print(error_msg)
            raise ValueError(error_msg)
            
        # self.config.logger.info(
        #     f"Начат поиск в хранилище с запросом: '{query[:50]}...'", 
        #     extra={"service": self.config.service}
        # )
        print(f"Начат поиск в хранилище с запросом: '{query[:50]}...'")
        
        try:
            # Используем similarity_search_with_score для получения оценок
            results_with_scores = self.vector_store.similarity_search_with_score(
                query, 
                k=top_k
            )
            
            search_results = []
            for document, distance in results_with_scores:
                # Конвертируем расстояние в сходство
                similarity = self._convert_distance_to_similarity(distance)
                
                if similarity >= score_threshold:
                    search_results.append(
                        SearchResult(
                            chunk_text=document.page_content,
                            source=document.metadata.get("source", ""),
                            page=document.metadata.get("page", -1),
                            index=document.metadata.get("index", -1),
                            header=document.metadata.get("header", -1),
                            chunk=document.metadata.get("chunk", -1),
                            content=document.metadata.get("content", -1),
                            html_path=document.metadata.get("html_path", -1),
                            similarity_score=similarity
                        )
                    )
            
            # self.config.logger.info(
            #     f"Поиск завершен, найдено {len(search_results)} результатов", 
            #     extra={"service": self.config.service}
            # )
            print(f"Поиск завершен, найдено {len(search_results)} результатов")
            
            return search_results
            
        except Exception as e:
            error_msg = f"Ошибка при выполнении поиска: {str(e)}"
            # self.config.logger.error(error_msg, extra={"service": self.config.service})
            print(error_msg)
            raise RuntimeError(error_msg)

    def _convert_distance_to_similarity(self, distance: float) -> float:
        """
        Конвертирует расстояние в оценку сходства.
        Для косинусного расстояния: similarity = 1 - distance

        Для косинусного расстояния в ChromaDB:
            distance = 1 - cosine_similarity
            similarity = 1 - distance = cosine_similarity
        """
        # Ограничиваем значение от 0 до 1
        return max(0.0, min(1.0, 1.0 - distance))

    def get_collection_info(self) -> Dict[str, Any]:
        """Возвращает информацию о коллекции."""
        try:
            count = self.collection.count()
            return {
                "collection": self.collection_name,
                "vectors_count": count,
                "embedding_device": self.device,
                "model": str(self.model_path.name),
                "db_path": str(self.db_path)
            }
        except Exception as e:
            # self.config.logger.error(
            #     f"Ошибка получения информации о коллекции: {e}", 
            #     extra={"service": self.config.service}
            # )
            print(f"Ошибка получения информации о коллекции: {e}")
            return {}

    def clear_collection(self) -> None:
        """Очищает всю коллекцию."""
        try:
            # Получаем все ID через векторное хранилище
            all_data = self.collection.get()
            if all_data['ids']:
                self.vector_store.delete(ids=all_data['ids'])
                # self.config.logger.info(
                #     f"Коллекция '{self.collection_name}' очищена", 
                #     extra={"service": self.config.service}
                # )
                print(f"Коллекция '{self.collection_name}' очищена")
        except Exception as e:
            error_msg = f"Ошибка очистки коллекции: {e}"
            # self.config.logger.error(error_msg, extra={"service": self.config.service})
            print(error_msg)
            raise RuntimeError(error_msg)
        
    def check_gpu_usage(self) -> Dict[str, Any]:
        """Возвращает информацию об использовании GPU"""
        if self.device == "cuda":
            return {
                "device": torch.cuda.get_device_name(0),
                "memory_allocated": f"{torch.cuda.memory_allocated() / 1024**2:.2f} MB",
                "memory_reserved": f"{torch.cuda.memory_reserved() / 1024**2:.2f} MB"
            }
        return {"status": "CPU used"}

<>:21: SyntaxWarning: invalid escape sequence '\m'
<>:21: SyntaxWarning: invalid escape sequence '\m'
C:\Users\AIshared\AppData\Local\Temp\ipykernel_26536\4201572280.py:21: SyntaxWarning: invalid escape sequence '\m'
  self.model_path = "D:\models\e5-lagre" # parent_folder / "model" / "e5-lagre"


In [24]:
chroma_storage = ChromaStorage("chroma_cad_db6")

GPU доступен: NVIDIA GeForce RTX 3090 Ti
Embedding функция инициализирована на cuda
Chroma хранилище инициализировано для коллекции 'vectors'
Инициализация успешна


In [25]:
start_time = time.time()

chroma_storage.add_vectors(
    ids=ids_list, 
    texts=documents,
    metadata=metadatas
)

response_time = time.time() - start_time
print(f"Время заполнения: {response_time:.3f} секунды")

Начато добавление векторов в хранилище
Успешно добавлено 2616 векторов в хранилище
Время заполнения: 14.088 секунды


## RAG

In [26]:
class RAG:
    """Класс для получения ответов от RAG"""

    def __init__(self, llm: LLM, storage: ChromaStorage):
        self.storage = storage
        self.llm = llm

    def get_rag_answer(self, question) -> str:
        """Запрос к векторному хранилищу и генерация ответа"""

        storage_reply = self.storage.search(question)
        context = storage_reply[0].content
        print(f"\nКонтекст:\n{context}")

        system_pronmt = 'Ты cпециалист системе автоматизированного проектирования Сарус. Отвечай на основании предоставленного контекста.'
        self.llm.set_system_prompt(system_pronmt)
        
        prompt = f"Как cпециалист системе автоматизированного проектирования Сарус ответь на ВОПРОС пользователя на основании КОНТЕКСТА.\n" \
                 f"ВОПРОС:\n{question}\n" \
                 f"КОНТЕКСТ:\n{context}"
        
        answer = self.llm.generate_answer(prompt)

        return answer

In [27]:
storage_reply = chroma_storage.search("Как построить фаску?")
storage_reply[0].content

Начат поиск в хранилище с запросом: 'Как построить фаску?...'
Поиск завершен, найдено 5 результатов


'Программный модуль CAD предусматривает следующие способы создания фаски между двумя линиями:\n\n1. «Симметричная фаска»;\n\n\n2. «Фаска по смещениям»;\n\n\n3. «Фаска по углу». Вызов команды «Фаска» осуществляется из панели «Лента» (вкладка «Чертёж», группа «Линии»). Выбор необходимого способа построения фаски может быть осуществлён из выпадающего списка под пиктограммой команды на панели «Лента»\xa0(рис.\xa0Рисунок).\n\nВызов команды «Фаска» [Изображение 296: ]\nРисунок\n\nВыбор способа построения фаски осуществляется при помощи соответствующих пиктограмм в диалоге параметров команды. В зависимости от выбранного способа изменяется набор управляющих параметров. В диалоге параметров предусмотрены поля выбираемых линий, между которыми необходимо построить фаску.\n\nПостроение симметричной фаски предполагает задание одного значения смещения (рис.\xa0Рисунок).\n\nДиалог команды «Фаска»\n\n\n[Изображение 297: ]\nРисунок Построение фаски по смещениям предполагает задание значений смещения вд

In [28]:
print(storage_reply[0].content)

Программный модуль CAD предусматривает следующие способы создания фаски между двумя линиями:

1. «Симметричная фаска»;


2. «Фаска по смещениям»;


3. «Фаска по углу». Вызов команды «Фаска» осуществляется из панели «Лента» (вкладка «Чертёж», группа «Линии»). Выбор необходимого способа построения фаски может быть осуществлён из выпадающего списка под пиктограммой команды на панели «Лента» (рис. Рисунок).

Вызов команды «Фаска» [Изображение 296: ]
Рисунок

Выбор способа построения фаски осуществляется при помощи соответствующих пиктограмм в диалоге параметров команды. В зависимости от выбранного способа изменяется набор управляющих параметров. В диалоге параметров предусмотрены поля выбираемых линий, между которыми необходимо построить фаску.

Построение симметричной фаски предполагает задание одного значения смещения (рис. Рисунок).

Диалог команды «Фаска»


[Изображение 297: ]
Рисунок Построение фаски по смещениям предполагает задание значений смещения вдоль каждой выбранной линии отде

In [29]:
rag = RAG(llm, chroma_storage)

In [30]:
question = "Как построить фаску?"

answer = rag.get_rag_answer(question)

print(f"\nОтвет:\n{answer}")

Начат поиск в хранилище с запросом: 'Как построить фаску?...'
Поиск завершен, найдено 5 результатов

Контекст:
Программный модуль CAD предусматривает следующие способы создания фаски между двумя линиями:

1. «Симметричная фаска»;


2. «Фаска по смещениям»;


3. «Фаска по углу». Вызов команды «Фаска» осуществляется из панели «Лента» (вкладка «Чертёж», группа «Линии»). Выбор необходимого способа построения фаски может быть осуществлён из выпадающего списка под пиктограммой команды на панели «Лента» (рис. Рисунок).

Вызов команды «Фаска» [Изображение 296: ]
Рисунок

Выбор способа построения фаски осуществляется при помощи соответствующих пиктограмм в диалоге параметров команды. В зависимости от выбранного способа изменяется набор управляющих параметров. В диалоге параметров предусмотрены поля выбираемых линий, между которыми необходимо построить фаску.

Построение симметричной фаски предполагает задание одного значения смещения (рис. Рисунок).

Диалог команды «Фаска»


[Изображение 297: ]

In [31]:
question = "Что такое изолиния?"

answer = rag.get_rag_answer(question)

print(f"\nОтвет:\n{answer}")

Начат поиск в хранилище с запросом: 'Что такое изолиния?...'
Поиск завершен, найдено 5 результатов

Контекст:
[Изображение 1174: ]
Команда «Изолинии поверхности»  предназначена для отображения сетки изопараметрических линий поверхности или их набора. Запуск команды осуществляется из панели «Лента» (вкладка «Параметры», группа «Анализ геометрии»). После запуска команды отображается диалог (рис. Рисунок).

Диалоговое окно команды «Изолинии поверхности»


[Изображение 1175: ]
Рисунок Список «Грани» содержит набор выбранных поверхностей, изопараметрические линии которых нужно отобразить.

Поля «Число линий по U/V» предназначены для заданий количества отображаемых изолиний в соответствующем направлении. В результате выполнения команды сетка изолиний сохраняется на поверхности в виде специального вспомогательного объекта «Эпюра изолиний», размещённого в папке «Измерители» «Навигатора модели».

Ответ:
 Изолинии — это линии, которые отображаются на поверхности с помощью специальной команды для

## Оценка семантического поиска

### Генерация тестового набора вопросов

In [32]:
class TestQAGenegator:
    """Класс генерации тестовых вопросов"""

    def __init__(self, data: pd.DataFrame, llm: LLM):
        self.data = data
        self.data["Question"] = ""
        self.llm = llm

    def questions_generation(self) -> pd.DataFrame:
        """Метод генерации тестовых вопросов"""

        for i in range(data.shape[0]):
            system_pronmt = 'Ты создаёшь вопросы к тексту для оценки семантического поиска. Выводи только вопрос не добавляй лишних слов'
            self.llm.set_system_prompt(system_pronmt)
            
            prompt = f"Задай содержательный вопрос по существу написанного в ТЕКСТЕ для оценки семантическо поиска\n" \
                     f"ТЕКСТ:\n{data.iloc[i, 1]}"
            # print(prompt)
            
            answer = self.llm.generate_answer(prompt)
            # print(answer)
            self.data.loc[i, "Question"] = answer

        return self.data

    def get_data(self) -> pd.DataFrame:
        return self.data

In [33]:
data = pd.read_csv("merged_cr3.csv", sep=";")
data.head()

,Header,Text,CleanText,HTML,Tables,Images,HTML_name
0,1 Назначение программного модуля CAD,Программный модуль CAD разработан на базе инте...,NaN,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",[],[],cf2981e0-0d00-48d2-9206-750c69b11912.html
1,2 Условия выполнения программного модуля CAD,Достаточными условиями выполнения программного...,NaN,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",[],[],fb0a33c0-9972-4761-856c-394425f09c0a.html
2,3.1 Запуск программного модуля CAD,Загрузка и запуск программного модуля CAD осущ...,NaN,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",[],"[{'id': 1, 'caption': '', 'path': WindowsPath(...",ae4eecfc-de30-4bba-879a-260ab2349ed7.html
3,3.2 Управление приложениями,\n[Изображение 2: ]\nДанное окно предназначено...,NaN,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",[],"[{'id': 2, 'caption': '', 'path': WindowsPath(...",3aef9b38-9e25-446a-ae24-3b41d655cda6.html
4,3.3 Работа с документами,Перечень инструментов для работы с документами...,NaN,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",[],[],c419d806-1a68-4943-8b85-dd0ac27a0361.html


In [34]:
q_generator = TestQAGenegator(data, llm)

In [35]:
start = time.time()

q_data = q_generator.questions_generation()

end = time.time()
print(f"Время исполнения: {(end - start):.3f} с.")

Время исполнения: 66.369 с.


In [36]:
data.head()

,Header,Text,CleanText,HTML,Tables,Images,HTML_name,Question
0,1 Назначение программного модуля CAD,Программный модуль CAD разработан на базе инте...,NaN,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",[],[],cf2981e0-0d00-48d2-9206-750c69b11912.html,Какие возможности предоставляет программный м...
1,2 Условия выполнения программного модуля CAD,Достаточными условиями выполнения программного...,NaN,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",[],[],fb0a33c0-9972-4761-856c-394425f09c0a.html,Какие минимальные технические характеристики ...
2,3.1 Запуск программного модуля CAD,Загрузка и запуск программного модуля CAD осущ...,NaN,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",[],"[{'id': 1, 'caption': '', 'path': WindowsPath(...",ae4eecfc-de30-4bba-879a-260ab2349ed7.html,Какие основные функциональные блоки содержит ...
3,3.2 Управление приложениями,\n[Изображение 2: ]\nДанное окно предназначено...,NaN,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",[],"[{'id': 2, 'caption': '', 'path': WindowsPath(...",3aef9b38-9e25-446a-ae24-3b41d655cda6.html,Какие действия можно совершить с дополнительн...
4,3.3 Работа с документами,Перечень инструментов для работы с документами...,NaN,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",[],[],c419d806-1a68-4943-8b85-dd0ac27a0361.html,Какие инструменты для работы с документами до...


In [37]:
data.tail()

,Header,Text,CleanText,HTML,Tables,Images,HTML_name,Question
203,12.9 Проверка модели,Команда «Проверка модели» предназначена для то...,NaN,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",[],"[{'id': 1176, 'caption': '', 'path': WindowsPa...",275d093a-031f-49ce-8ad8-fe13729480a3.html,Какие типы ошибок можно выявить с помощью ком...
204,13.1 Экспорт и импорт 3D-геометрии,\n[Изображение 1177: ]\nКоманды «Экспорт» и «...,NaN,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",[],"[{'id': 1177, 'caption': '', 'path': WindowsPa...",98f3a2ea-381f-4bc1-a36a-92535ebb5ea4.html,Какие параметры доступны в диалоге экспорта в...
205,13.2 Печать,\n[Изображение 1182: ]\nКоманда «Печать» вызы...,NaN,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",[],"[{'id': 1182, 'caption': '', 'path': WindowsPa...",b1d403a1-b74e-4917-b6c4-64973cc78864.html,Какие параметры можно настроить при печати до...
206,13.3 Экспорт PDF,\n[Изображение 1185: ]\nКоманда «Сохранить PDF...,NaN,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",[],"[{'id': 1185, 'caption': '', 'path': WindowsPa...",519c0530-51ea-4fc9-96b8-2c0a8b797922.html,"Какие шаги необходимо выполнить для того, что..."
207,13.4 Экспорт изображения,\n[Изображение 1188: ]\nКоманда «Экспорт изобр...,NaN,"<!DOCTYPE html>\n<html lang=""ru"">\n<head>\n ...",[],"[{'id': 1188, 'caption': '', 'path': WindowsPa...",5bc1bf72-07aa-4a7d-9130-b9af7f80cff0.html,Какие параметры доступны для настройки при эк...


In [38]:
data.to_csv("merged_cr3_qa.csv", sep=";", index=False)

### Оцека поиска в RAG

In [39]:
class SemanticSearchEvaluation:

    def __init__(
        self,
        emb_path: str
        
    ):
        # Загружаем эмбеддинги
        self.emb_path = emb_path

        # Проверка доступности GPU
        self.device = self._get_best_device()
        
        self.embedding_function = HuggingFaceEmbeddings(
            model_name=self.emb_path,
            model_kwargs={'device': self.device},
            encode_kwargs={
                'normalize_embeddings': True,
                'convert_to_tensor': True
            },
            show_progress=False
        )
        self.result = dict()
        self.collection_name = "vectors"

    def _get_best_device(self) -> str:
        """Определяет оптимальное устройство для вычислений (GPU/CPU)."""
        if torch.cuda.is_available():
            print(f"GPU доступен: {torch.cuda.get_device_name(0)}")
            return "cuda"
        else:
            print("GPU недоступен, используется CPU")
            return "cpu"
            
    def evaluate(self,
                 db_path: str,
                 data: pd.DataFrame,
                 column_name: str,
                 displacement: int=0):
        # Загружаем базу Chroma
        # Создаём клиента
        parent_folder = Path(os.getcwd()) # Path(__file__).parent.parent
        path = str(parent_folder / db_path)
        
        self.client = PersistentClient(path=path)
        
        # Инициализация векторного хранилища с embedding функцией
        self.vector_store = Chroma(
            client=self.client,
            collection_name=self.collection_name,
            embedding_function=self.embedding_function,
            collection_metadata={"hnsw:space": "cosine"}
        )
        
        # Получаем коллекцию для прямых операций
        self.collection = self.vector_store._collection

        # датасет
        df = data

        total = 0    # Вопросов всего
        correct = 0  # Правильных индексов (первый верный)
        both = 0   # Оба индекса верные
        present = 0  # Индекс есть в списке из 5

        for i in tqdm(range(displacement, df.shape[0])):
            if df.iloc[i][column_name] == "-" or df.iloc[i][column_name] != df.iloc[i][column_name]:
                continue
                
            curr_indexes = []
            vect_res = self.vector_store.similarity_search(
                df.iloc[i][column_name],
                k=5
            )
            len_vec = len(vect_res)
            for j in range(len_vec):
                curr_indexes.append(int(vect_res[j].metadata['Index']))
            total += 1
            if len(curr_indexes) and i == curr_indexes[0]:
                correct += 1
            if len(curr_indexes) and (i == curr_indexes[0] or i == curr_indexes[1]):
                both += 1
            if i in curr_indexes:
                present += 1

        self.result["total"] = total
        self.result["correct"] = correct
        self.result["both"] = both
        self.result["present"] = present
        self.result["top1"] = self.result["correct"]/self.result["total"]*100
        self.result["top2"] = self.result["both"]/self.result["total"]*100
        self.result["top5"] = self.result["present"]/self.result["total"]*100

        return self.result

    def show_result(self):
        print(f'Вопросов всего: {self.result["total"]}')
        print(f'Правильных индексов (первый верный): {self.result["correct"]} доля: {self.result["top1"]:.2f}%')
        print(f'Первый или второй верные: {self.result["both"]} доля: {self.result["top2"]:.2f}%')
        print(f'Индекс есть в списке из 5: {self.result["present"]} доля: {self.result["top5"]:.2f}%')

In [41]:
retriver_test = SemanticSearchEvaluation("D:\models\e5-lagre")

res = retriver_test.evaluate("chroma_cad_db6", data, "Question")
res

<>:1: SyntaxWarning: invalid escape sequence '\m'
<>:1: SyntaxWarning: invalid escape sequence '\m'
C:\Users\AIshared\AppData\Local\Temp\ipykernel_26536\1854761222.py:1: SyntaxWarning: invalid escape sequence '\m'
  retriver_test = SemanticSearchEvaluation("D:\models\e5-lagre")


GPU доступен: NVIDIA GeForce RTX 3090 Ti


100%|██████████| 208/208 [00:02<00:00, 78.40it/s]


{'total': 208,
 'correct': 194,
 'both': 204,
 'present': 206,
 'top1': 93.26923076923077,
 'top2': 98.07692307692307,
 'top5': 99.03846153846155}

In [42]:
retriver_test.show_result()

Вопросов всего: 208
Правильных индексов (первый верный): 194 доля: 93.27%
Первый или второй верные: 204 доля: 98.08%
Индекс есть в списке из 5: 206 доля: 99.04%
